In [266]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn import neighbors

In [267]:
properties = pd.read_csv('/home/santiago/Desktop/Organizacion De Datos (75.06)/TP2/trainPreProcess.csv', low_memory=False)
test_set = pd.read_csv('/home/santiago/Desktop/Organizacion De Datos (75.06)/TP2/testPreProcess.csv', low_memory=False)

In [268]:
#Uno los dataframes para hacer clustering de TODO
all_df = test_set.append(properties)
#all_df.info()
len(all_df)

143009

In [269]:
#Transformo la fecha a un formato aprovechable para el algoritmo de clustering
all_df['created_on'] = pd.to_datetime(all_df.created_on)
all_df['created_on'] = all_df['created_on'].dt.strftime('%Y%m%d')
all_df['created_on'] = pd.to_numeric(all_df.created_on)


all_df['lat'] = all_df['lat'].apply(lambda x: x*2000)
all_df['lon'] = all_df['lon'].apply(lambda x: x*2000)
all_df['distancia_obelisco'] = all_df['distancia_obelisco'].apply(lambda x: x*2000)
all_df['distancia_minima_subte'] = all_df['distancia_minima_subte'].apply(lambda x: x*2000)
all_df['distancia_minima_privado'] = all_df['distancia_minima_privado'].apply(lambda x: x*2000)
all_df['distancia_minima_publico'] = all_df['distancia_minima_publico'].apply(lambda x: x*2000)




In [270]:
#Saco los precios 
train = all_df.loc[:, ['place_name_encoded','surface_covered_in_m2','lat','lon','distancia_minima_subte']]

#room,surface_total_in_m2,properti_type y place_name

#train = all_df.loc[:, ['property_type_encoded', 'rooms','lat','lon','surface_covered_in_m2']]


#train = all_df.loc[:, ['property_type_encoded', 'rooms','surface_total_in_m2','place_name_encoded']]


In [271]:
#Ahora si empieza K-Means
kmeans = KMeans(init='k-means++', n_clusters= 1100, n_init=1)#150 y 10
cluster_prediction = kmeans.fit(train).predict(train) 
cluster_prediction = cluster_prediction.flatten()
train['cluster'] = cluster_prediction

In [272]:
train_clusters = train['cluster'][len(test_set):]
test_clusters = train['cluster'][:len(test_set)]

#Agrego los clusters a los datasets
properties['cluster'] = train_clusters
test_set['cluster'] = test_clusters
#properties[properties['cluster'] == 0]['surface_total_in_m2'].value_counts()

In [273]:
#test_set[test_set['cluster'] == 0]['surface_total_in_m2'].value_counts()

In [274]:
#Ahora que tengo todo separado en clusters, me interesa calcular el precio por cluster como el promedio
price_by_cluster = pd.DataFrame()
price_by_cluster['price_usd'] = properties.groupby(['cluster'])['price'].transform('mean').values
properties['price_usd'] = price_by_cluster['price_usd']
price_by_cluster = properties[['cluster','price_usd']].copy()
price_by_cluster.drop_duplicates(inplace=True)
price_by_cluster = price_by_cluster.sort_values(by='cluster',ascending=True)

In [289]:
prediction_on_train = properties[['price' , 'price_usd']]
score_KMeans = mean_squared_error(prediction_on_train['price'].values, prediction_on_train['price_usd'].values)
score_KMeans

362678308382.04889

In [290]:
prediction_on_test = pd.merge(test_set,price_by_cluster,on='cluster',how = 'left')
prediction_on_test = prediction_on_test[['id','price_usd']]
prediction_on_test.index = prediction_on_test['id']
prediction_on_test = prediction_on_test[['price_usd']]

In [291]:
#prediction_on_test.to_csv('K_Means-25-11.csv')

In [292]:
prediction_on_test.dropna(inplace=True)
len(prediction_on_test)

14166

In [293]:
prediction_on_train.head(100)

,price,price_usd
0,"1,199,000.00","190,228.92"
1,"2,000,000.00","444,149.76"
2,"570,000.00","820,576.72"
3,"500,000.00","820,576.72"
4,"450,000.00","162,233.33"
5,"135,000.00","131,366.67"
6,"180,000.00","172,355.93"
7,"77,000.00","218,264.00"
8,"570,000.00","190,157.87"
9,"98,500.00","577,065.71"


In [294]:
properties[properties['cluster'] == 300]

,Unnamed: 0,id,created_on,property_type_encoded,place_name_encoded,state_name_encoded,lat,lon,surface_covered_in_m2,surface_total_in_m2,floor,rooms,price,distancia_obelisco,distancia_minima_subte,distancia_minima_privado,distancia_minima_publico,cluster,price_usd
10484,15910,b43021309a8a819b2c160fef6f3e455f2766a940,2015-09-30,1,414,2,-34.81,-58.45,180.00,285.00,0.00,4.00,"130,000.00",0.27,0.17,0.04,0.07,300,"165,584.78"
11490,17334,71a1bef3dcafac0b70ba5d8131a21eb43d6f0659,2015-11-10,1,398,2,-34.77,-58.44,160.00,300.00,0.00,2.00,"135,000.00",0.23,0.15,0.04,0.07,300,"165,584.78"
22939,35806,53e9646e91e3f72f2d57851d6426541272014672,2016-10-01,1,392,2,-34.81,-58.43,214.00,214.00,0.00,2.00,"160,000.00",0.26,0.19,0.04,0.07,300,"165,584.78"
24805,38545,64053ae0d0a57872952c4fff806ad7740df34563,2016-10-29,1,414,2,-34.80,-58.45,200.00,313.00,0.00,2.00,"120,000.00",0.26,0.17,0.04,0.07,300,"165,584.78"
25456,39562,22406de9d49ffcfc0137d21b5dfa726c5963e25a,2016-11-06,1,398,2,-34.80,-58.43,231.00,197.00,0.00,2.00,"160,000.00",0.24,0.17,0.04,0.07,300,"165,584.78"
27459,42354,b33c691932dc286862537667dd62f2446dd928ac,2016-11-23,1,414,2,-34.81,-58.45,170.00,268.00,0.00,2.00,"112,000.00",0.27,0.17,0.04,0.07,300,"165,584.78"
31088,47841,a0e1c7af1a060e7509aca1d94d75214729860abe,2017-01-05,1,414,2,-34.80,-58.45,159.00,227.00,0.00,5.00,"119,000.00",0.26,0.16,0.04,0.07,300,"165,584.78"
36126,55083,a6efa78eddcccaf4f1bcc9354b162f4065c85726,2017-02-14,1,414,2,-34.79,-58.45,200.00,"1,650.00",0.00,2.00,"300,000.00",0.26,0.16,0.04,0.07,300,"165,584.78"
47505,72662,3d67485b92894ed9181c0367e82913c66e8670d9,2017-05-09,2,285,2,-34.83,-58.46,181.50,270.00,0.00,2.00,"285,000.00",0.30,0.18,0.04,0.07,300,"165,584.78"
49763,76022,3f3a3916f67bfd9279d918422bb3e8c02058698d,2017-05-19,2,285,2,-34.79,-58.47,181.50,270.00,0.00,2.00,"285,000.00",0.27,0.15,0.04,0.07,300,"165,584.78"


In [295]:
median_absolute_error(prediction_on_train['price'].values, prediction_on_train['price_usd'].values)

89577.261410788371

In [296]:
len(properties)

128843

In [297]:
properties_KNN = properties[abs(properties['price_usd'] - properties['price']) > 80000]
properties_KMeans = properties[abs(properties['price_usd'] - properties['price']) <= 80000]

pd.options.display.float_format = '{:20,.2f}'.format
len(properties_KNN)

69718

In [298]:
properties_KNN = properties_KNN[['id','cluster','place_name_encoded','surface_covered_in_m2','lat','lon','distancia_minima_subte','price']]
properties_KNN

,id,cluster,place_name_encoded,surface_covered_in_m2,lat,lon,distancia_minima_subte,price
0,0de2467efb2476545515f109b506e084728ea6a2,471,439,120.00,-34.68,-58.73,0.30,"1,199,000.00"
1,1649a3c35359d681abae4a63d9a8545054d2096b,215,470,76.00,-34.59,-58.43,0.01,"2,000,000.00"
2,496af9a18172b64a3b271b59c749a66e5e388704,401,498,140.00,-34.61,-58.36,0.01,"570,000.00"
3,dee414c7198dd58137279f0afc56551481535cdc,401,498,140.00,-34.61,-58.37,0.01,"500,000.00"
4,1d141a2cb833184512533525bc1a8a449b494172,80,192,110.00,-34.65,-58.66,0.21,"450,000.00"
7,4b3e9a3b36feceace001bde7da7f08d849586372,66,552,65.00,-34.53,-58.72,0.27,"77,000.00"
8,825bdda1207b7a5026eed2d61064541116738e11,249,452,51.50,-34.56,-58.72,0.24,"570,000.00"
9,f175ebe7af799642974dbe1f7b48abef1478eae9,202,47,265.00,-34.42,-58.66,0.32,"98,500.00"
11,0750777bf2f84c68120db625a771f519cd1ee291,586,511,105.00,-34.59,-58.39,0.01,"150,000.00"
15,660bfbc863d0ec1d4f4a8d7ea07fa4ba50f23017,66,552,65.00,-34.54,-58.72,0.26,"90,000.00"
